In [1]:
import pandas as pd
import re
import numpy as np
from hanspell import spell_checker
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

In [116]:
df_n_p=pd.read_csv('네이버페이_플레이스토어.csv')
df_k_p=pd.read_csv('카카오페이_플레이스토어.csv')
df_s_p=pd.read_csv('삼성페이_플레이스토어.csv')
df_n_a=pd.read_csv('네이버페이_앱스토어.csv')
df_k_a=pd.read_csv('카카오페이_앱스토어.csv')


In [118]:
#네이버
naver=pd.merge(df_n_p,df_n_a,how='outer')
naver=naver.sort_values(['DATE'],ascending=False)
naver.reset_index(drop=True,inplace=True)

#카카오
kakao=pd.merge(df_k_p,df_k_a,how='outer')
kakao=kakao.sort_values(['DATE'],ascending=False)
kakao.reset_index(drop=True,inplace=True)

#삼성
samsung=df_s_p

In [120]:
kakao

,DATE,STAR,REVIEW
0,2022-05-20 03:10:17,1,주민득록증이 없는디 어캐 민증을 발급기간을 입력하라는건지...
1,2022-05-19 23:02:15,5,페이중에는 짱인듯
2,2022-05-19 22:53:53,1,지문 변경한 적 없는데 지문 변경됐다고 재등록하라고 나와요. 그래서 재등록 계속 하...
3,2022-05-19 22:05:08,3,비밀 최형옥 성선정
4,2022-05-19 10:54:18,2,Mymine7@18
...,...,...,...
4425,2019-05-28 18:36:01,3,아직 카톡에서 벗어나지 못해서 거기서 처리하는 메뉴들도 많네요. 차차 해줄거라고 믿...
4426,2019-05-28 18:33:48,5,간결하니 좋네요
4427,2019-05-28 17:32:35,5,카카오페이 독립축하합니다
4428,2019-05-28 16:28:27,5,너무 간편하고 UI가 손에 편하네요 이런 금융앱은 처음이에요


In [121]:
samsung= samsung[(samsung.DATE.str[:4]=='2022') | (samsung.DATE.str[:4]=='2021')|(samsung.DATE.str[:4]=='2020')]

In [122]:
kakao=kakao[(kakao.DATE.str[:4]=='2022') | (kakao.DATE.str[:4]=='2021')|(kakao.DATE.str[:4]=='2020')]

In [127]:
print('기간 조정 후 네이버페이 리뷰수:',len(naver))
print('기간 조정 후 삼성페이 리뷰수:',len(samsung))
print('기간 조정 후 카카오페이 리뷰수:',len(kakao))

기간 조정 후 네이버페이 리뷰수: 662
기간 조정 후 삼성페이 리뷰수: 47419
기간 조정 후 카카오페이 리뷰수: 4106


# 전처리 방법
1) 한글만 남기기 <br>
2) 띄어쓰기 재진행 <br>
3) 맞춤법 검사 <br>
4) 반복되는 단어 ex) 굿굿 -> 굿으로 바꾸기  => 작동을 잘 안하는 듯
* 1~3 부정  
* 4~5 : 긍정

In [8]:
for data in [kakao,naver,samsung]:

    data['cleaning_REVIEW'] = data.REVIEW.str.replace("[^가-힣 ]","") #한글, 공백만 남기기
    data['cleaning_REVIEW'] = data['cleaning_REVIEW'].str.replace('^ +', "") #emptyvalue로 변경
    data['cleaning_REVIEW'].replace('', np.nan, inplace=True) #nan으로 변경
    data.dropna(axis=0,inplace=True) 

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set

In [9]:
#py-hanspell 사용 => 맞춤법 검사 및 띄어쓰기
for data in [kakao,naver,samsung]:
    lists=[]
    for i in range(len(data)):
        check=spell_checker.check(data.cleaning_REVIEW.iloc[i]).checked
        lists.append(check)
    data.cleaning_REVIEW=lists
    
dfkakao=kakao
dfnaver=naver
dfsamsung=samsung

C:\Users\user\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [132]:
dfnaver.head(50)

,DATE,STAR,REVIEW,cleaning_REVIEW,token_REVIEW
0,2022-05-20 00:13:02,4,결제 방식을 바코드 결제가 아니라 페이코처럼 삼성 페이로(nfc 방식으로) 결제할 ...,결제 방식을 바코드 결제가 아니라 페이코처럼 삼성 피로 방식으로 결제할 수 있게 업...,"[결제, 방식, 바코드, 결제, 페이, 삼성, 방식, 결제, 업데이트, 주시, 좋다]"
1,2022-05-17 16:49:48,5,굿,굿,[]
2,2022-05-16 03:26:43,1,2단계 인증 로그인 설정시 로그인때 몇번씩 반복해도 알림이 오지않고 OTP번호로도 ...,단계 인증 로그인 설정 시 로그인 때 몇 번씩 반복해도 알림이 오지 않고 번호로도 ...,"[단계, 인증, 로그인, 설정, 로그인, 반복, 알림, 오지, 호로도, 맞다, 로그..."
3,2022-05-14 17:46:40,1,웹보다 접근성 ui 다떨어짐 쓸이유가 없는 앱,웹보다 접근성 다 떨어짐 쓸 이유가 없는 앱,"[접근성, 떨어지다, 이유]"
4,2022-05-14 08:56:01,3,맨처음에 앱 다운로드하고 본인인증을 하라 해서 했더니 인증번호가 안 옵니다... 몇...,맨 처음에 앱 다운로드하고 본인인증을 하라 해서 했더니 인증번호가 안 옵니다 몇 번...,"[처음, 다운로드, 본인, 인증, 인증, 번호, 해도, 절대, 깔다, 지우다, 스팸..."
5,2022-05-13 17:58:57,1,네이버 페이 QR결제 .. 이거 한번 오류나면 다시는 못하고 지웠다 깔아야함.,네이버 페이 결제 이거 한번 오류 나면 다시는 못하고 지웠다 깔아야 함,"[네이버, 페이, 결제, 한번, 오류, 다시다, 지우다, 깔다]"
6,2022-05-13 16:28:44,2,네이버 아이디가 있을 경우 앱을 시작할수 없어요..실명인증된 아이디를 모두 탈퇴해야...,네이버 아이디가 있을 경우 앱을 시작할 수 없어요 실명인증된 아이디를 모두 탈퇴해야...,"[네이버, 아이디, 경우, 시작, 실명, 인증, 아이디, 모두, 탈퇴, 때문]"
7,2022-05-12 19:17:08,1,본인인증 하는데 인증번호가 뜨지 않더라구요,본인인증하는데 인증번호가 뜨지 않더라고요,"[본인, 인증, 인증, 번호, 뜨다]"
8,2022-05-12 13:22:49,5,굿,굿,[]
9,2022-05-12 01:39:16,1,인증 하려하는데 문자가 안오네요,인증하려 하는데 문자가 안 오네요,"[인증, 문자]"


In [128]:
dfsamsung

,DATE,STAR,REVIEW,cleaning_REVIEW,token_REVIEW
0,2022-05-20 12:38:26,5,"많은분들이 공감하시겠지만 ""편하다"" 이죠. 그럼 어떻게 편하냐? 간단한 본인인증후 ...",많은 분들이 공감하시겠지만 편하다 오죠 그럼 어떻게 편하냐 간단한 본인인증 후 많은...,"[많다, 공감, 편하다, 편하다, 간단하다, 본인, 인증, 많다, 매장, 간편, 결..."
1,2022-05-20 12:23:35,5,항상 잘 쓰는중,항상 잘 쓰는 중,"[항상, 쓰다]"
2,2022-05-20 11:56:03,5,잘 사용하고 있어요,잘 사용하고 있어요,[사용]
3,2022-05-20 11:40:17,5,편해요,편해요,[편하다]
4,2022-05-20 11:22:38,5,사용하기 참 편해요^^,사용하기 참 편해요,"[사용, 편하다]"
...,...,...,...,...,...
47414,2020-01-01 05:20:49,5,편리해서 좋와요,편리해서 좋아요,"[편리하다, 좋다]"
47415,2020-01-01 04:39:19,4,쓸만합니다,쓸만합니다,[]
47416,2020-01-01 01:25:30,5,너무편해용,너무 편해요,[편하다]
47417,2020-01-01 01:18:54,5,부족함 없이 편하게 사용할수 있어요,부족함 없이 편하게 사용할 수 있어요,"[부족함, 편하다, 사용]"


In [42]:
dfkakao.isnull().sum()

DATE               0
STAR               0
REVIEW             0
cleaning_REVIEW    0
dtype: int64

In [59]:
pd.read_csv('카카오페이_전처리.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4028 entries, 0 to 4027
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   DATE             4028 non-null   object
 1   STAR             4028 non-null   int64 
 2   REVIEW           4028 non-null   object
 3   cleaning_REVIEW  4025 non-null   object
 4   token_REVIEW     4028 non-null   object
dtypes: int64(1), object(4)
memory usage: 157.5+ KB


In [36]:
from konlpy.tag import  Okt
okt=Okt()

In [43]:
# dfkakao['token_REVIEW']=dfkakao.cleaning_REVIEW.map(lambda x : okt.morphs(x,norm=True,stem=True)) #원형으로
# dfsamsung['token_REVIEW']=dfsamsung.cleaning_REVIEW.map(lambda x : okt.morphs(x,norm=True,stem=True))
# dfnaver['token_REVIEW']=dfnaver.cleaning_REVIEW.map(lambda x : okt.morphs(x,norm=True,stem=True))

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [47]:
def pos_filtering(text): 
    pos_word_list = okt.pos(text, stem = True) #원형으로
    pos_list = ['Noun', 'Verb', 'Adjective'] #명사 - 동사 - 형용사 추출
    pos_filtered_word_list = []

    for word, pos in pos_word_list: # 품사필터링
        if pos in pos_list:
            pos_filtered_word_list.append(word)

    return pos_filtered_word_list

In [96]:
dfkakao['token_REVIEW']=dfkakao.cleaning_REVIEW.map(lambda x : pos_filtering(x))
dfsamsung['token_REVIEW']=dfsamsung.cleaning_REVIEW.map(lambda x : pos_filtering(x))
dfnaver['token_REVIEW']=dfnaver.cleaning_REVIEW.map(lambda x : pos_filtering(x))

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [97]:
#한글자 지우기
dfkakao['token_REVIEW'] = dfkakao['token_REVIEW'].map(lambda x :[word for word in x if len(word) != 1])
dfsamsung['token_REVIEW'] = dfsamsung['token_REVIEW'].map(lambda x :[word for word in x if len(word) != 1])
dfnaver['token_REVIEW'] = dfnaver['token_REVIEW'].map(lambda x :[word for word in x if len(word) != 1])

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [101]:
#불용어제거
stopwords = ['하다','있다','되어다','그렇다','되다','없다','안되다','해주다','아니다','가능하다','돼다','같다','주시','해보다','수가','보이다','차다','피로','정말','진짜',
             '써다','싶다','어떻다','자다','보다','이다','이렇다','나다','오다','가다','하라','하나요','쓸다','않다','이제','갑자기']
dfkakao['token_REVIEW'] = dfkakao['token_REVIEW'].map(lambda x : [word for word in x if word not in stopwords])
dfsamsung['token_REVIEW'] = dfsamsung['token_REVIEW'].map(lambda x : [word for word in x if word not in stopwords])
dfnaver['token_REVIEW'] = dfnaver['token_REVIEW'].map(lambda x : [word for word in x if word not in stopwords])

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [102]:
word_list = sum(dfsamsung['token_REVIEW'], [])

from collections import Counter
c = Counter(word_list)
c.most_common(80)

[('좋다', 13609),
 ('편하다', 12012),
 ('편리하다', 5472),
 ('사용', 4443),
 ('쓰다', 3244),
 ('페이', 3054),
 ('삼성', 2990),
 ('최고', 2254),
 ('카드', 2185),
 ('지갑', 2165),
 ('결제', 1674),
 ('아주', 1345),
 ('다니다', 1254),
 ('굳다', 1204),
 ('갤럭시', 1151),
 ('간편하다', 1013),
 ('들다', 921),
 ('아이폰', 871),
 ('필요', 811),
 ('이유', 799),
 ('때문', 798),
 ('교통카드', 727),
 ('등록', 611),
 ('매우', 604),
 ('빠르다', 461),
 ('기능', 457),
 ('인식', 447),
 ('오류', 422),
 ('유용하다', 416),
 ('불편하다', 395),
 ('신세계', 391),
 ('가지', 369),
 ('굿굿', 362),
 ('뜨다', 361),
 ('핸드폰', 357),
 ('편리', 352),
 ('삼베', 347),
 ('처음', 343),
 ('하나', 343),
 ('다시', 326),
 ('역시', 320),
 ('그냥', 316),
 ('결재', 314),
 ('어플', 302),
 ('나오다', 296),
 ('만족하다', 292),
 ('바꾸다', 291),
 ('개편', 285),
 ('넘어오다', 281),
 ('계속', 279),
 ('업데이트', 276),
 ('많다', 268),
 ('다른', 260),
 ('쓰기', 258),
 ('쉬다', 255),
 ('지문', 252),
 ('광고', 250),
 ('이용', 248),
 ('휴대폰', 247),
 ('세상', 234),
 ('항상', 230),
 ('사람', 226),
 ('감사하다', 223),
 ('만들다', 210),
 ('살다', 207),
 ('괜찮다', 202),
 ('모르다', 200),
 ('애플', 196),

In [115]:
dfnaver

,DATE,STAR,REVIEW,cleaning_REVIEW,token_REVIEW
0,2022-05-20 00:13:02,4,결제 방식을 바코드 결제가 아니라 페이코처럼 삼성 페이로(nfc 방식으로) 결제할 ...,결제 방식을 바코드 결제가 아니라 페이코처럼 삼성 피로 방식으로 결제할 수 있게 업...,"[결제, 방식, 바코드, 결제, 페이, 삼성, 방식, 결제, 업데이트, 주시, 좋다]"
1,2022-05-17 16:49:48,5,굿,굿,[]
2,2022-05-16 03:26:43,1,2단계 인증 로그인 설정시 로그인때 몇번씩 반복해도 알림이 오지않고 OTP번호로도 ...,단계 인증 로그인 설정 시 로그인 때 몇 번씩 반복해도 알림이 오지 않고 번호로도 ...,"[단계, 인증, 로그인, 설정, 로그인, 반복, 알림, 오지, 호로도, 맞다, 로그..."
3,2022-05-14 17:46:40,1,웹보다 접근성 ui 다떨어짐 쓸이유가 없는 앱,웹보다 접근성 다 떨어짐 쓸 이유가 없는 앱,"[접근성, 떨어지다, 이유]"
4,2022-05-14 08:56:01,3,맨처음에 앱 다운로드하고 본인인증을 하라 해서 했더니 인증번호가 안 옵니다... 몇...,맨 처음에 앱 다운로드하고 본인인증을 하라 해서 했더니 인증번호가 안 옵니다 몇 번...,"[처음, 다운로드, 본인, 인증, 인증, 번호, 해도, 절대, 깔다, 지우다, 스팸..."
...,...,...,...,...,...
657,2021-08-03 12:38:12,5,굿굿굿~bbb,굿굿굿,[굿굿굿]
658,2021-08-03 11:10:32,5,야호 네이버 페이 앱이네요,야호 네이버 페이 앱이네요,"[야호, 네이버, 페이]"
659,2021-08-03 00:32:59,5,편하고 좋아요!,편하고 좋아요,"[편하다, 좋다]"
660,2021-08-01 21:53:31,5,빨리만들어주세요,빨리 만들어주세요,[만들다]


In [114]:
dfnaver[dfnaver.token_REVIEW=='[]']

,DATE,STAR,REVIEW,cleaning_REVIEW,token_REVIEW


In [113]:
# dfkakao.to_csv('카카오페이_전처리.csv',index=False,encoding='utf-8-sig')
# dfnaver.to_csv('네이버페이_전처리.csv',index=False,encoding='utf-8-sig')
# dfsamsung.to_csv('삼성페이_전처리.csv',index=False,encoding="utf-8-sig")